# Features transformation

in [1_Combine_features.py](1_Combine_features.py) we combined separate raw files with features into csv files by features types. Here we further transform features and combine them into one csv file. 

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import pandas.tseries as ts

plt.style.use('ggplot')

## Equity, Commodities, and Fixed Income returns

We calculate weekly returns of equity, commodity, and fixed income indices.

### Equities

In [32]:
##  Equities

# Read csv
df = pd.read_csv('./data/raw/equities.csv', index_col=0)
df.head()

,RLG,RIY,MXEF,RTY,RLV,SML,MID,MXEU,SPX
date,,,,,,,,,
1999-12-31,837.618,768.101,489.42,505.397,568.782,197.79,444.67,133.69,1469.25
2000-01-03,839.040,761.457,496.22,498.122,555.658,194.31,438.13,132.91,1455.22
2000-01-04,799.930,731.400,500.38,479.035,539.401,187.39,422.54,127.72,1399.42
2000-01-05,798.684,733.296,491.09,479.457,543.944,188.18,426.37,124.55,1402.11
2000-01-06,780.350,728.364,486.04,475.892,552.570,187.02,424.45,123.95,1403.45


In [43]:
# Calculate weekly volatility for future use
df_v = df.copy()

# Calculate returns
df_v = (df_v - df_v.shift(1))/df_v.shift(1)

# Create week variable
df_v['week'] = pd.to_datetime(df_v.index).map(lambda x: x.isocalendar()[0]*100 + x.isocalendar()[1])

# Calculate weekly volatility
volat = df_v.groupby('week').agg(np.std) * np.sqrt(5) # Convert to weekly volatility

volat.columns = [name+'_volat' for name in volat.columns]

volat.head()

,RLG_volat,RIY_volat,MXEF_volat,RTY_volat,RLV_volat,SML_volat,MID_volat,MXEU_volat,SPX_volat
week,,,,,,,,,
199952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200001,0.077252,0.059081,0.032830,0.053508,0.050024,0.051664,0.054465,0.052134,0.052865
200002,0.040747,0.029618,0.038452,0.043543,0.019451,0.039871,0.049832,0.027427,0.025546
200003,0.008705,0.009017,0.030512,0.012882,0.013143,0.008871,0.004334,0.024834,0.008103
200004,0.039889,0.032237,0.017538,0.023199,0.028159,0.019259,0.026985,0.028622,0.034163


In [44]:
# Calculate weekly return

# Resample to weekly
df.index = pd.to_datetime(df.index)
df_week = df.resample('W').last()
df_week = (df_week - df_week.shift(1))/df_week.shift(1)

#Index weeks
df_week.index = pd.to_datetime(df_week.index).map(lambda x: x.isocalendar()[0]*100 + x.isocalendar()[1])
df_week.head()



,RLG,RIY,MXEF,RTY,RLV,SML,MID,MXEU,SPX
date,,,,,,,,,
199952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200001,-0.024939,-0.019264,0.005476,-0.032422,-0.011326,-0.029931,-0.017743,-0.051836,-0.018908
200002,0.028575,0.022270,0.047815,0.039410,0.013591,0.044978,0.026695,0.037946,0.016428
200003,-0.002307,-0.011046,-0.015787,0.051833,-0.023245,0.033217,0.009879,-0.023790,-0.016237
200004,-0.064911,-0.054364,-0.016631,-0.054849,-0.039322,-0.060243,-0.050081,0.008720,-0.056336


In [49]:
# Combine returns and volatility
df = df_week.join(volat)
df = df.iloc[1:,:]

# Save result
df.to_csv('./data/equities.csv')

df.head()

,RLG,RIY,MXEF,RTY,RLV,SML,MID,MXEU,SPX,RLG_volat,RIY_volat,MXEF_volat,RTY_volat,RLV_volat,SML_volat,MID_volat,MXEU_volat,SPX_volat
date,,,,,,,,,,,,,,,,,,
200001,-0.024939,-0.019264,0.005476,-0.032422,-0.011326,-0.029931,-0.017743,-0.051836,-0.018908,0.077252,0.059081,0.032830,0.053508,0.050024,0.051664,0.054465,0.052134,0.052865
200002,0.028575,0.022270,0.047815,0.039410,0.013591,0.044978,0.026695,0.037946,0.016428,0.040747,0.029618,0.038452,0.043543,0.019451,0.039871,0.049832,0.027427,0.025546
200003,-0.002307,-0.011046,-0.015787,0.051833,-0.023245,0.033217,0.009879,-0.023790,-0.016237,0.008705,0.009017,0.030512,0.012882,0.013143,0.008871,0.004334,0.024834,0.008103
200004,-0.064911,-0.054364,-0.016631,-0.054849,-0.039322,-0.060243,-0.050081,0.008720,-0.056336,0.039889,0.032237,0.017538,0.023199,0.028159,0.019259,0.026985,0.028622,0.034163
200005,0.065199,0.045177,0.032822,0.041381,0.017087,0.033234,0.034799,0.025625,0.047208,0.021114,0.021776,0.029559,0.033362,0.030866,0.030957,0.016759,0.033353,0.023506
